.# 오픈 데이터 추가
- KorQuAD 2.1의 전체 데이터는 47,957 개의 Wikipedia article에 대해 102,960 개의 질의응답 쌍으로, Training set 83,486 개, Dev set 10,165 개의 질의응답쌍으로 구분
- KorQuAD v2.1의 원본 데이터가 아닌 하이퍼링크, 태그, 유니코드 BOM를 제거하여 전처리를 하였고, context 토큰 길이가 1,156이 넘어간 데이터들은 제외
- 답변 이상한 것들("고문서로 발견된 유물이 원어로는 존재하지 않고 번역된 언어로만 존재할 경우") 제외

In [1]:
from datasets import load_dataset

/data/ephemeral/home/sh/level2-mrc-nlp-08/.venv_sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("KorQuAD/squad_kor_v2")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answer', 'url', 'raw_html'],
        num_rows: 83486
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answer', 'url', 'raw_html'],
        num_rows: 10165
    })
})

In [4]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

In [16]:
df_train = train_dataset.to_pandas()
df_validation = validation_dataset.to_pandas()

In [6]:
import re
import unicodedata

def preprocess_text(text):
    # 유니코드 BOM 제거
    text = text.lstrip('\ufeff')
    
    # 하이퍼링크 제거 (http, https, www 형식의 링크 제거)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    
    # 여러 개의 공백을 하나의 공백으로 변환
    text = re.sub(r'\s+', ' ', text)
    
    # 유니코드 문자가 정상적으로 인코딩되었는지 확인 후 정규화
    text = unicodedata.normalize('NFC', text)
    
    # 양쪽 끝 공백 제거
    text = text.strip()
    
    return text

# 예시 텍스트
text = '''\ufeff안녕하세요! <a href="https://example.com">여기를 클릭하세요</a> www.example.com에서 더 많은 정보를 확인하세요.'''
clean_text = preprocess_text(text)

print(clean_text)

안녕하세요! 더 많은 정보를 확인하세요.


In [7]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")  # BertTokenizer

In [8]:
import numpy as np

In [9]:
'''
id                                                       8089
title                                                     예고범
context     <!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...
question                                   드라마 예고범의 감독은 누구일까?
answer      {'text': '나카무라 요시히로, 히라바야시 카츠토시, 사와다 메구미', 'an...
url                         https://ko.wikipedia.org/wiki/예고범
raw_html    <!DOCTYPE html>\n<html class="client-nojs" dir...
Name: 0, dtype: object
'''

train_filter_lst = []
for _, row in df_train.iterrows():
    context = row['context']
    preprocess_context = preprocess_text(context)
    if len(tokenizer.encode(preprocess_context)) > 1500:
        continue

    # question = row['question']
    # question = preprocess_text(question)
    # row['question'] = question
    answer = row['answer']
    text = answer['text']
    # if text[-1] in ['다', '것', '"', '.', '우', '에', '문', '해']:
    if text[-1] in ['다', '것', '"', '.', '우', '문']:
        continue
    
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]
    if text != find_text:
        continue    
    try:
        del answer['html_answer_start']
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answer'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 2.1_' + str(row['id'])
    train_filter_lst.append(row)

Token indices sequence length is longer than the specified maximum sequence length for this model (4330 > 4096). Running this sequence through the model will result in indexing errors


In [10]:
len(train_filter_lst)

3720

In [11]:
import pandas as pd
df_train_filter = pd.DataFrame(train_filter_lst)

In [13]:
df_train_filter.to_csv('koquadv2_train.csv', encoding='utf-8-sig')

In [17]:
df_train_filter

,id,title,context,question,answer,url,raw_html,html
1,KorQuAD 2.1_36615,심규언,"심규언 - 위키백과, 우리 모두의 백과사전 심규언 위키백과, 우리 모두의 백과사전....",심규언은 17대 지방 선거에서 몇 표를 득표하였는가?,"{'text': ['20,890 표'], 'answer_start': [1209]}",https://ko.wikipedia.org/wiki/심규언,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
19,KorQuAD 2.1_65016,변태_(식물),"변태 (식물) - 위키백과, 우리 모두의 백과사전 변태 (식물) 위키백과, 우리 모...",감자는 식물의 어느 부분이 변태한 거야?,"{'text': ['땅속줄기'], 'answer_start': [657]}",https://ko.wikipedia.org/wiki/변태_(식물),"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
36,KorQuAD 2.1_47235,올림픽_스노보드,"올림픽 스노보드 - 위키백과, 우리 모두의 백과사전 올림픽 스노보드 위키백과, 우리...",동계 올림칙 스노보드 부분에서 미국이 가져간 총 메달의 개수는 모두 몇개입니까?,"{'text': ['24'], 'answer_start': [181]}",https://ko.wikipedia.org/wiki/올림픽_스노보드,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
45,KorQuAD 2.1_74239,야크_존,"야크 존 - 위키백과, 우리 모두의 백과사전 야크 존 위키백과, 우리 모두의 백과사...",야크 존의 국적은?,"{'text': ['네덜란드'], 'answer_start': [84]}",https://ko.wikipedia.org/wiki/야크_존,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
48,KorQuAD 2.1_66506,변환_색인_버퍼,"변환 색인 버퍼 - 위키백과, 우리 모두의 백과사전 변환 색인 버퍼 위키백과, 우리...",가상 주소를 사용했을때 메모리 접근시간이 두배로 늘어나는\n단점을 보완하기 위해 도...,"{'text': ['TLB'], 'answer_start': [113]}",https://ko.wikipedia.org/wiki/변환_색인_버퍼,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
...,...,...,...,...,...,...,...,...
77835,KorQuAD 2.1_99999,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...","페르소나는 마케팅,디자인,판매등 다양한 조직간 사용자 이해를 위한 어떠한 도구로써의...","{'text': ['상호소통의 도구'], 'answer_start': [1497]}",https://ko.wikipedia.org/wiki/페르소나_(방법론),"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
77836,KorQuAD 2.1_100000,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나가 다양한 조직에서 사용자 이해를 위해 어떠한 도구로 이용될 수 있을까?,"{'text': ['상호소통의 도구'], 'answer_start': [1497]}",https://ko.wikipedia.org/wiki/페르소나_(방법론),"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
77837,KorQuAD 2.1_100001,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나의 사용이 어떤 개발을 할 때 중요한 이점을 가지는 연구방법인가?,"{'text': ['제품 개발'], 'answer_start': [433]}",https://ko.wikipedia.org/wiki/페르소나_(방법론),"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
77838,KorQuAD 2.1_100002,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나는 다양한 조직간에 어떤 도구로 사용할 수 있나?,"{'text': ['상호소통의 도구'], 'answer_start': [1497]}",https://ko.wikipedia.org/wiki/페르소나_(방법론),"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None


In [18]:
validation_filter_lst = []
for _, row in df_validation.iterrows():
    context = row['context']
    preprocess_context = preprocess_text(context)
    if len(tokenizer.encode(preprocess_context)) > 1500:
        continue

    # question = row['question']
    # question = preprocess_text(question)
    # row['question'] = question
    answer = row['answer']
    text = answer['text']
    if text[-1] in ['다', '것', '"', '.', '우', '문']:
        continue
    
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]
    if text != find_text:
        continue    
    try:
        del answer['html_answer_start']
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answer'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 2.1_' + str(row['id'])
    validation_filter_lst.append(row)

In [19]:
len(validation_filter_lst)

448

In [20]:
df_validation_filtering = pd.DataFrame(validation_filter_lst)

In [21]:
import pandas as pd
df_validation_filtering.to_csv('koquadv2_val.csv', encoding='utf-8-sig')

In [22]:
df_validation_filtering

,id,title,context,question,answer,url,raw_html,html
37,KorQuAD 2.1_69079,교토세이카다이마에_역,"교토세이카다이마에 역 - 위키백과, 우리 모두의 백과사전 교토세이카다이마에 역 위키...",교토세이카다이마에역은 무인역이지만 언제 담당자가 배치될까?,"{'text': ['대학의 성수기(입학식이나 수험 시)나 아침 러시아워'], 'an...",https://ko.wikipedia.org/wiki/교토세이카다이마에_역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
38,KorQuAD 2.1_37433,김상훈_(1963년),"김상훈 (1963년) - 위키백과, 우리 모두의 백과사전 김상훈 (1963년) 위키...",김상훈이 제33회 행정고시에 합격한 연도는 언제일까?,"{'text': ['1989년'], 'answer_start': [499]}",https://ko.wikipedia.org/wiki/김상훈_(1963년),"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
45,KorQuAD 2.1_79142,철산군,"철산군 - 위키백과, 우리 모두의 백과사전 철산군 위키백과, 우리 모두의 백과사전....",철산군은 어떤 기후를 가지고 있는가?,"{'text': ['해양성 기후'], 'answer_start': [455]}",https://ko.wikipedia.org/wiki/철산군,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
73,KorQuAD 2.1_36491,비야베르데_알토_역,"비야베르데 알토 역 - 위키백과, 우리 모두의 백과사전 비야베르데 알토 역 위키백과...",비야베르데 알토역에서 마르가리타스로 가려면 어떤 지하철을 이용해야할까?,"{'text': ['C4호선'], 'answer_start': [454]}",https://ko.wikipedia.org/wiki/비야베르데_알토_역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
91,KorQuAD 2.1_36951,중계역,"중계역 - 위키백과, 우리 모두의 백과사전 중계역 위키백과, 우리 모두의 백과사전....",2010년 중계역의 7호선 승차고객수는 몇명입니까?,"{'text': ['15,817'], 'answer_start': [1207]}",https://ko.wikipedia.org/wiki/중계역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
...,...,...,...,...,...,...,...,...
10052,KorQuAD 2.1_107738,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",박근혜가 SNS 여론 조작 보고를 받은 사실이 공개된 해는?,"{'text': ['2012년'], 'answer_start': [1026]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10053,KorQuAD 2.1_107739,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 보고회에서 만든 조직의 명칭은 무엇인가?,"{'text': ['새마음포럼'], 'answer_start': [1145]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10054,KorQuAD 2.1_107740,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",새누리당 당원이 발행인으로 있는 출간지의 이름은 무엇인가?,"{'text': ['국정일보'], 'answer_start': [1313]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10055,KorQuAD 2.1_107741,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 조직한 조직으로 SNS 여론 조작을 위해 결성한 조직은?,"{'text': ['새마음포럼'], 'answer_start': [1145]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None


In [23]:
from datasets import load_from_disk
datasets = load_from_disk("../resources/data/train_dataset")

In [24]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [25]:
raw_train_df = datasets['train'].to_pandas()

In [26]:
df_train_filter = df_train_filter.drop(['raw_html', 'html'], axis=1)
df_train_filter = df_train_filter.rename(columns={'answer': 'answers'})
raw_train_df_augmentation = pd.concat([raw_train_df, df_train_filter])
raw_train_df_augmentation = raw_train_df_augmentation.reset_index(drop=True)

In [27]:
raw_train_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__,url
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293.0,42.0,NaN
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638.0,2873.0,NaN
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028.0,230.0,NaN
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146.0,992.0,NaN
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334.0,548.0,NaN
...,...,...,...,...,...,...,...,...
7667,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...","페르소나는 마케팅,디자인,판매등 다양한 조직간 사용자 이해를 위한 어떠한 도구로써의...",KorQuAD 2.1_99999,"{'text': ['상호소통의 도구'], 'answer_start': [1497]}",NaN,NaN,https://ko.wikipedia.org/wiki/페르소나_(방법론)
7668,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나가 다양한 조직에서 사용자 이해를 위해 어떠한 도구로 이용될 수 있을까?,KorQuAD 2.1_100000,"{'text': ['상호소통의 도구'], 'answer_start': [1497]}",NaN,NaN,https://ko.wikipedia.org/wiki/페르소나_(방법론)
7669,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나의 사용이 어떤 개발을 할 때 중요한 이점을 가지는 연구방법인가?,KorQuAD 2.1_100001,"{'text': ['제품 개발'], 'answer_start': [433]}",NaN,NaN,https://ko.wikipedia.org/wiki/페르소나_(방법론)
7670,페르소나_(방법론),"페르소나 (방법론) - 위키백과, 우리 모두의 백과사전 페르소나 (방법론) 위키백과...",페르소나는 다양한 조직간에 어떤 도구로 사용할 수 있나?,KorQuAD 2.1_100002,"{'text': ['상호소통의 도구'], 'answer_start': [1497]}",NaN,NaN,https://ko.wikipedia.org/wiki/페르소나_(방법론)


In [28]:
from datasets import Dataset

In [29]:
train_dataset = Dataset.from_pandas(raw_train_df_augmentation, preserve_index=False)

In [30]:
raw_validation_df = datasets['validation'].to_pandas()

In [31]:
raw_validation_df

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184
...,...,...,...,...,...,...,...
235,전단 (제나라),"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"{'answer_start': [1084], 'text': ['제 양왕']}",53265,329
236,티칼,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장...",12190,1378
237,윤치호,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"{'answer_start': [247], 'text': ['미나미 지로']}",5326,2031
238,신잉카국,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"{'answer_start': [563], 'text': ['200,000명']}",48575,1963


In [32]:
df_validation_filtering = df_validation_filtering.drop(['raw_html', 'html'], axis=1)
df_validation_filtering = df_validation_filtering.rename(columns={'answer': 'answers'})
raw_validation_df_augmentation = pd.concat([raw_validation_df, df_validation_filtering])
raw_validation_df_augmentation = raw_validation_df_augmentation.reset_index(drop=True)
raw_validation_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__,url
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027.0,2146.0,NaN
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765.0,3106.0,NaN
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694.0,1180.0,NaN
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536.0,141.0,NaN
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991.0,184.0,NaN
...,...,...,...,...,...,...,...,...
683,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",박근혜가 SNS 여론 조작 보고를 받은 사실이 공개된 해는?,KorQuAD 2.1_107738,"{'text': ['2012년'], 'answer_start': [1026]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
684,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 보고회에서 만든 조직의 명칭은 무엇인가?,KorQuAD 2.1_107739,"{'text': ['새마음포럼'], 'answer_start': [1145]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
685,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",새누리당 당원이 발행인으로 있는 출간지의 이름은 무엇인가?,KorQuAD 2.1_107740,"{'text': ['국정일보'], 'answer_start': [1313]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
686,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 조직한 조직으로 SNS 여론 조작을 위해 결성한 조직은?,KorQuAD 2.1_107741,"{'text': ['새마음포럼'], 'answer_start': [1145]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈


In [33]:
validation_dataset = Dataset.from_pandas(raw_validation_df_augmentation, preserve_index=False)

In [34]:
from datasets import DatasetDict

In [35]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'url'],
        num_rows: 7672
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'url'],
        num_rows: 688
    })
})


In [37]:
dataset_dict.save_to_disk('../resources/data_kosquadv2_train_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 688/688 [00:00<00:00, 89193.62 examples/s]
